In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import pandas as pd
import random
import os
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math
from sklearn.multioutput import MultiOutputRegressor
import matplotlib.pyplot as plt # 득점모델 변수 중요도
import seaborn as sns
%matplotlib inline
from sklearn.decomposition import PCA
from statsmodels.stats.outliers_influence import variance_inflation_factor
import xgboost as xgb
from sklearn.model_selection import KFold

In [ ]:
pip install statsmodels

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [5]:
train_x = train_df.filter(regex='X') # Input : X Featrue
train_y = train_df.filter(regex='Y') # Output : Y_01 Feature..
test_x = test_df.filter(regex='X')
test_y = test_df.filter(regex='Y')

In [6]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(train_x.values, i) for i in range(train_x.shape[1])]
vif["features"] = train_x.columns
vif

c:\Users\User\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\regression\linear_model.py:1736: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


,VIF Factor,features
0,3.338393,X_01
1,1.008929,X_02
2,2.185961,X_03
3,0.000000,X_04
4,1.766929,X_05
5,2.345513,X_06
6,1.024752,X_07
7,1.117746,X_08
8,1.177054,X_09
9,5.767093,X_10


In [81]:
remove_vif = vif[vif["VIF Factor"] >8]

In [82]:
rmlist = list(remove_vif.features.unique())
rmlist.extend(["X_04", "X_47", 'X_48', "X_10","X_11"])

In [83]:
rmlist

['X_13',
 'X_14',
 'X_15',
 'X_17',
 'X_18',
 'X_42',
 'X_43',
 'X_45',
 'X_04',
 'X_47',
 'X_48',
 'X_10',
 'X_11']

In [84]:
train_x.drop(rmlist,inplace=True,axis=1)
test_x.drop(rmlist,inplace=True,axis=1)

C:\Users\User\AppData\Local\Temp\ipykernel_13120\2183760222.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_x.drop(rmlist,inplace=True,axis=1)
C:\Users\User\AppData\Local\Temp\ipykernel_13120\2183760222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x.drop(rmlist,inplace=True,axis=1)


In [7]:
from sklearn.preprocessing import StandardScaler
train_x = StandardScaler().fit_transform(train_x)
train_x = pd.DataFrame(data = train_x)

In [8]:
xgb_model = xgb.XGBRegressor(n_estimators=100, max_depth=20, learning_rate=0.1, n_jobs=-1)
kfold = KFold(n_splits=5, shuffle=True)
kfold_regr = MultiOutputRegressor(xgb_model)

In [9]:
loss_fold = []

for train, test in kfold.split(train_x):
    XGB = MultiOutputRegressor(xgb_model).fit(train_x.loc[train], train_y.loc[train])
    train_predict = XGB.predict(train_x.loc[test])
    scores = math.sqrt(mean_squared_error(train_predict, train_y.loc[test]))
    loss_fold.append(scores)

In [13]:
ftr_importances_values = XGB.feature_importances_
ftr_importances = pd.Series(ftr_importances_values, index=train_x.columns)
ftr_top12 = ftr_importances.sort_values(ascending=False)[20:]
 
plt.figure(figsize=(8, 6))
sns.barplot(x=ftr_top12, y=ftr_top12.index)
plt.show()
l = ftr_top12.index

AttributeError: 'MultiOutputRegressor' object has no attribute 'feature_importances_'

In [10]:
loss_fold

[1.2125756775250882,
 1.2602899676282933,
 1.2224376351335162,
 1.2139617905463396,
 1.2390518066740726]

In [130]:
from sklearn.model_selection import GridSearchCV
 
param_grid = [
        {'max_depth' : [5,6,7,8],  'learning_rate':[0.01, 0.1], 'random_state':[42], "n_jobs":[-1]}
        
    ]
forest_reg = MultiOutputRegressor(xgb.XGBRegressor())
 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True,
                           n_jobs = -1)
 
grid_search.fit(train_x, train_y)

ValueError: Invalid parameter 'learning_rate' for estimator MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_to_onehot=None,
                          